In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
eng_characters = [chr(i) for i in range(ord('a'), ord('z')+1)]  # English lowercase letters
eng_characters += [chr(i) for i in range(ord('A'), ord('Z')+1)]  # English uppercase letters
eng_characters += [chr(i) for i in range(ord('0'), ord('9')+1)]  # Digits
eng_characters += ['?','<','>']
# Add other characters as needed, such as punctuation marks, special symbols, etc.

print(eng_characters)
# Define the range of Unicode code points for Telugu characters
TELUGU_START = 0x0C00
TELUGU_END = 0x0C7F

# Generate all Telugu characters
telugu_characters = [chr(code_point) for code_point in range(TELUGU_START, TELUGU_END + 1)]
telugu_characters += ['?','<','>']

print(telugu_characters)



['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '<', '>']
['ఀ', 'ఁ', 'ం', 'ః', 'ఄ', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఌ', '\u0c0d', 'ఎ', 'ఏ', 'ఐ', '\u0c11', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', '\u0c29', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'ఴ', 'వ', 'శ', 'ష', 'స', 'హ', '\u0c3a', '\u0c3b', '\u0c3c', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', '\u0c45', 'ె', 'ే', 'ై', '\u0c49', 'ొ', 'ో', 'ౌ', '్', '\u0c4e', '\u0c4f', '\u0c50', '\u0c51', '\u0c52', '\u0c53', '\u0c54', 'ౕ', 'ౖ', '\u0c57', 'ౘ', 'ౙ', 'ౚ', '\u0c5b', '\u0c5c', '\u0c5d', '\u0c5e', '\u0c5f', 'ౠ', 'ౡ', 'ౢ', 'ౣ', '\u0c64', '\u0c65', '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯'

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_train.csv'
valid_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_valid.csv'
# Load train and validation datasets

# Load the datasets
train_dataset = pd.read_csv(train_file)
valid_dataset = pd.read_csv(valid_file)

# Extract the source and target columns
source_data = train_dataset.iloc[:, 0].values
target_data = train_dataset.iloc[:, 1].values


In [4]:
source_data = ['<' + s + '>' for s in source_data]
target_data = ['<' + t + '>' for t in target_data]

In [5]:
loc = 1
e,t = train_dataset.iloc[loc]
char_to_index_eng = {char: i for i, char in enumerate(eng_characters)}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}
def char_to_tensor_eng(word, char_to_index_eng):
    return torch.tensor([char_to_index_eng[char] for char in word], dtype=torch.long)

print(e)
print(char_to_tensor_eng(e,char_to_index_eng))

char_to_index_tel = {char: i for i, char in enumerate(telugu_characters)}
index_to_char_tel = {i: char for char, i in char_to_index_tel.items()}
def char_to_tensor_tel(word, char_to_index_tel):
    return torch.tensor([char_to_index_tel[char] for char in word], dtype=torch.long)

print(t)
t1 = char_to_tensor_tel(t,char_to_index_tel)
print(t1)

characters = [telugu_characters[char_int.item()] for char_int in t1]
string = ''.join(characters)
# Print the characters
print("Characters:", string)
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(f"Batch {batch_idx}:")
#     print("Data shape:", data.shape)
#     print("Target shape:", target.shape)
#     # You can print more details about the data and target if needed
#     break  # This will break after printing the first batch

factamfos
tensor([ 5,  0,  2, 19,  0, 12,  5, 14, 18])
ఫ్యాక్టమ్ఫోస్
tensor([43, 77, 47, 62, 21, 77, 31, 46, 77, 43, 75, 56, 77])
Characters: ఫ్యాక్టమ్ఫోస్


In [7]:

source = [char_to_tensor_eng(s, char_to_index_eng) for s in source_data]
target = [char_to_tensor_tel(s, char_to_index_tel) for s in target_data]

In [9]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
# Pad the tensors to the same length

source_padded = pad_sequence(source, batch_first=True, padding_value=62)
target_padded = pad_sequence(target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(source_padded.size(1), target_padded.size(1))

# Pad source and target to the same length
source_padded = torch.nn.functional.pad(source_padded, (0, max_length - source_padded.size(1)), value=62)
target_padded = torch.nn.functional.pad(target_padded, (0, max_length - target_padded.size(1)), value=128)

# Add 62 at the beginning and 63 at the end of each item in source_padded
# source_padded = torch.cat((torch.full((source_padded.shape[0], 1), 63, dtype=torch.long), 
#                            source_padded, torch.full((source_padded.shape[0], 1), 64, dtype=torch.long)), dim=1)
# target_padded = torch.cat((torch.full((target_padded.shape[0], 1), 129, dtype=torch.long), 
#                            target_padded, torch.full((target_padded.shape[0], 1), 130, dtype=torch.long)), dim=1)

In [10]:
target_padded[0]

tensor([129,  53,  56,  77,  36,  62,  38,  63,  48,  62, 130, 128, 128, 128,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128])

In [11]:
def t2p(s,t):
    characters = [eng_characters[char_int.item()] for char_int in s]
    string1 = ''.join(characters)
#     print(string)
    characters = [telugu_characters[char_int.item()] for char_int in t]
    string2 = ''.join(characters)
    print(f"{string1} {len(string1)} : {string2} {len(string2)} :")
t2p(source_padded[0],target_padded[0])

<vastadira>??????????????????? 30 : <వస్తాదిరా>??????????????????? 30 :


In [12]:
# Create a TensorDataset
dataset = TensorDataset(source_padded, target_padded)

# Create a DataLoader
batch_size = 64
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage of the DataLoader
cnt=1
for batchid,(s,t) in enumerate(tqdm(train_dataloader)):
    #source_batch, target_batch = batch
    # Your training code here
    print(s[0])
    for item_s, item_t in zip(s, t):
#     print(f"{batchid}:")
        t2p(item_s,item_t)
#     print(s)
#     cnt+=1
#     if(cnt>2):break
    if(batchid>0): break


  0%|          | 1/800 [00:00<00:33, 23.82it/s]

tensor([63, 15,  0, 17,  8, 15,  0,  0, 11,  8, 18, 19,  0,  0, 21, 20, 64, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62])
<paripaalistaavu>????????????? 30 : <పరిపాలిస్తావు>??????????????? 30 :
<arisepalli>?????????????????? 30 : <అరిశేపల్లి>?????????????????? 30 :
<eruganidi>??????????????????? 30 : <ఎరుగనిది>???????????????????? 30 :
<vayasulokante>??????????????? 30 : <వయసులోకంటే>?????????????????? 30 :
<belgianlo>??????????????????? 30 : <బెల్జియన్లో>????????????????? 30 :
<gonjalo>????????????????????? 30 : <గొంజలో>?????????????????????? 30 :
<alaativaarilo>??????????????? 30 : <అలాటివారిలో>????????????????? 30 :
<connelly>???????????????????? 30 : <కాన్నేల్లీ>?????????????????? 30 :
<karrateesukoni>?????????????? 30 : <కర్రతీసుకొని>???????????????? 30 :
<aarojuraatri>???????????????? 30 : <ఆరోజురాత్రి>????????????????? 30 :
<marchukosagindi>????????????? 30 : <మార్చుకోసాగింది>????????????? 30 :
<ananthasubramanian>?????????? 30 : <అనంతసుబ్రహ్మణ్యం>???????????? 30 :

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, cell_type):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Character embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
    
    def forward(self, x):
        embedded = self.embedding(x)
        # embedded shape: (seq_length:max_length, N(batch size:32), embedding_size:100)
        output, (hidden,cell) = self.rnn(embedded)
        return hidden,cell

class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Devanagari character embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        
        # Decoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden, cell):
        # shape of x: (N) but required (1, N)
        x = x.unsqueeze(0)
        
        embedded = self.embedding(x)
        # embedded shape: (1, N, embedding_size)
        output, (hidden,cell) = self.rnn(embedded, (hidden,cell))
        # shape of outputs: (1, N, hidden_size)
        predictions = self.fc(output)
        # shape of predictionos:  (1, N, length of tel_char)
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_letter_size = 131
        
        outputs = torch.zeros(target_len, batch_size, target_letter_size).to(device)
        
        # Encoder forward pass
        hidden,cell = self.encoder(source)
        
        x = target[0]
        
#         # Decoder initial hidden state
#         decoder_hidden = encoder_hidden        
#         # Decoder input (start token)
#         decoder_input = torch.tensor([[SOS_token]] * batch_size)      
        # Decoder forward pass
    
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)
            x = target[t] if np.random.rand() < teacher_forcing_ratio else best_guess
        
        return outputs


In [14]:
input_size = len(char_to_index_eng)
print(input_size)
char_to_index_eng['?']

65


62

In [15]:
# Define the model architecture
from torch.utils.tensorboard import SummaryWriter
input_size = len(char_to_index_eng)
hidden_size = 300
embedding_size = 300
output_size = len(char_to_index_tel)
num_layers = 2
cell_type = 'LSTM'
LEARNING_RATE = 0.001
NUM_EPOCHS = 1

encoder_net = Encoder(input_size, embedding_size, hidden_size, num_layers, cell_type).to(device)
decoder_net = Decoder(output_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type).to(device)
model = Seq2Seq(encoder_net, decoder_net).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index = char_to_index_tel['?'])
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

writer = SummaryWriter(f'runs/Loss_plot')
step = 0
# Train the model
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batchid,(inp_data,target) in enumerate(tqdm(train_dataloader)):
        inp_data = inp_data.to(device)
        target = target.to(device)
#         print(inp_data.shape)
        output = model(inp_data, target)
        indices = output.argmax(2)
        # output shape: (trg_len, batch_size, output_dim)
#         print(output.shape, output.shape[0],output.shape[1], output.shape[2])
        if(batchid%400==0):
            t2p(inp_data[0],target[0])
            t2p(inp_data[0],indices[0])
            print(indices[0])
            #print(inp_data.shape, target.shape,output.shape, indices.shape)
        output = output[1:].reshape(-1, output.shape[2])

        target = target[1:].reshape(-1)

        # Flatten the output and target tensors to compute the loss
        optimizer.zero_grad()
        loss = criterion(output, target)
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
        optimizer.step()
        step+=1
        writer.add_scalar('Training loss', loss, global_step=step)


    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")

# # Evaluate the model on the validation dataset
# model.eval()
# total_val_loss = 0
# with torch.no_grad():
#     for input_tensor, target_tensor in valid_loader:
#         output = model(input_tensor, target_tensor)
#         output_dim = output.shape[-1]

#         # Flatten the output and target tensors to compute the loss
#         val_loss = criterion(output.view(-1, output_dim), target_tensor.view(-1))
#         total_val_loss += val_loss.item()

#     print(f"Validation Loss: {total_val_loss / len(valid_loader)}")


2024-05-03 19:27:49.255332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 19:27:49.255454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 19:27:49.387300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  0%|          | 0/800 [00:00<?, ?it/s]

<chudalenemonani>????????????? 30 : <చూడలేనేమోనని>???????????????? 30 :
<chudalenemonani>????????????? 30 : ఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀ 30 :
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')


 50%|█████     | 401/800 [00:34<00:33, 12.07it/s]

<edurupadaalani>?????????????? 30 : <ఎదురుపడాలని>????????????????? 30 :
<edurupadaalani>?????????????? 30 : ఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀఀ 30 :
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')


100%|██████████| 800/800 [01:08<00:00, 11.70it/s]

Epoch 1, Loss: 2.911114453077316
